In [1]:
import pandas as pd

In [2]:
get_ipython().run_line_magic('matplotlib','inline')

In [3]:
import matplotlib

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
r_cols = ['user_id','movie_id','rating','unix_timestamp']
ratings = pd.read_csv(r'C:\Users\hp\Anaconda3\ml-10M100K\ratings.dat',sep='::',names=r_cols,encoding='latin-1')
#print(ratings.head(5000),ratings.shape)
#ratings.sample(len(ratings))

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [7]:
ncv = int(ratings.shape[0]/100)
train_ratings = ratings[0:ncv]
test_ratings = ratings[ncv:2*ncv]
print(train_ratings.head(20))
print()
print(test_ratings.head(20))
print(train_ratings.shape,test_ratings.shape)

    user_id  movie_id  rating  unix_timestamp
0         1       122     5.0       838985046
1         1       185     5.0       838983525
2         1       231     5.0       838983392
3         1       292     5.0       838983421
4         1       316     5.0       838983392
5         1       329     5.0       838983392
6         1       355     5.0       838984474
7         1       356     5.0       838983653
8         1       362     5.0       838984885
9         1       364     5.0       838983707
10        1       370     5.0       838984596
11        1       377     5.0       838983834
12        1       420     5.0       838983834
13        1       466     5.0       838984679
14        1       480     5.0       838983653
15        1       520     5.0       838984679
16        1       539     5.0       838984068
17        1       586     5.0       838984068
18        1       588     5.0       838983339
19        1       589     5.0       838983778

        user_id  movie_id  rating

In [8]:
class MF():
    #initializing the user-movie rating matrix(R), no. of latent features(K), alpha and beta.
    def __init__(self,R,K,alpha,beta,iterations):
        self.R = R
        self.num_users,self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
    #initializing user-feature and movie-feature matrix
    def train(self):
        self.P = np.random.normal(scale=1./self.K,size=(self.num_users,self.K))
        self.Q = np.random.normal(scale=1./self.K,size=(self.num_items,self.K))
        
        #initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        #List of training samples
        self.samples = [
            (i,j,self.R[i,j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i,j] > 0
        ]
        
        #stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1)%20 == 0:
                print("Iteration: %d ; error = %.4f" %(i+1, mse))
        return training_process
    
    #computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x,y in zip(xs,ys):
            error += pow(self.R[x,y] - predicted[x,y],2)
        return (np.sqrt(error)/R.shape[0])
    
    #stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i,j,r in self.samples:
            prediction = self.get_rating(i,j)
            e= (r-prediction)
            self.b_u[i] += self.alpha*(e-self.beta*self.b_u[i])
            self.b_i[j] += self.alpha*(e-self.beta*self.b_i[j])
            
            self.P[i,:] += self.alpha*(e*self.Q[j,:]-self.beta*self.P[i,:])
            self.Q[j,:] += self.alpha*(e*self.P[i,:]-self.beta*self.Q[j,:])
    
    #ratings for user i and movie j
    def get_rating(self,i,j):
        prediction = self.b+self.b_u[i]+self.b_i[j]+self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    #full user-movie rating matrix
    def full_matrix(self):
        return mf.b+mf.b_u[:,np.newaxis]+mf.b_i[np.newaxis:,]+mf.P.dot(mf.Q.T)


In [9]:
R = np.array(train_ratings.pivot(index='user_id',columns='movie_id',values='rating').fillna(0))

In [10]:
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [4., 3., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 4., ..., 0., 0., 0.]])

In [11]:
mf = MF(R,K=20,alpha=0.001,beta=0.01,iterations=500)

In [12]:
training_process = mf.train()

Iteration: 20 ; error = 0.3779
Iteration: 40 ; error = 0.3683
Iteration: 60 ; error = 0.3619
Iteration: 80 ; error = 0.3549
Iteration: 100 ; error = 0.3445
Iteration: 120 ; error = 0.3302
Iteration: 140 ; error = 0.3146
Iteration: 160 ; error = 0.2997
Iteration: 180 ; error = 0.2862
Iteration: 200 ; error = 0.2744
Iteration: 220 ; error = 0.2640
Iteration: 240 ; error = 0.2551
Iteration: 260 ; error = 0.2473
Iteration: 280 ; error = 0.2405
Iteration: 300 ; error = 0.2345
Iteration: 320 ; error = 0.2292
Iteration: 340 ; error = 0.2245
Iteration: 360 ; error = 0.2204
Iteration: 380 ; error = 0.2167
Iteration: 400 ; error = 0.2133
Iteration: 420 ; error = 0.2103
Iteration: 440 ; error = 0.2076
Iteration: 460 ; error = 0.2052
Iteration: 480 ; error = 0.2030
Iteration: 500 ; error = 0.2010


In [13]:
new_R = mf.full_matrix()

In [19]:
R2 = np.array(test_ratings.pivot(index='user_id',columns='movie_id',values='rating').fillna(0))
R3 = R[:,:R2.shape[1]]
R2 = R2[:R3.shape[0],:]
print(R2,R2.shape,R3.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 3. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]] (730, 6128) (730, 6128)


In [20]:
import tensorflow as tf

In [25]:
h_size = R3.shape[0]
train_inp = tf.placeholder(shape=[R3.shape[0], R3.shape[1]], dtype=tf.float32)
train_out = tf.placeholder(shape=[R2.shape[0], R2.shape[1]], dtype=tf.float32)
print(train_inp,train_out)

Tensor("Placeholder_4:0", shape=(730, 6128), dtype=float32) Tensor("Placeholder_5:0", shape=(730, 6128), dtype=float32)


In [26]:
#weights_hidden = tf.Variable(tf.truncated_normal(shape=[new_R.shape[1], h_size], dtype=tf.float64))
weights_hidden = tf.Variable(tf.random_normal([1,h_size],
                                       mean=0,
                                       stddev=0.01,
                                       name="weights"))
bias_hidden = tf.Variable(tf.random_normal([h_size,R3.shape[1]],
                                    mean=0,
                                    stddev=0.01,
                                    name="bias"))
#bias_hidden = tf.Variable(tf.truncated_normal(shape=[1,h_size], dtype=tf.float64))

In [27]:
af_input_hidden = tf.matmul(train_inp, weights_hidden)+bias_hidden

ValueError: Dimensions must be equal, but are 6128 and 1 for 'MatMul_1' (op: 'MatMul') with input shapes: [730,6128], [1,730].

In [47]:
print(af_input_hidden)

Tensor("add_3:0", shape=(6128, 730), dtype=float32)


In [48]:
hidden_layer_output = tf.nn.sigmoid(af_input_hidden) 

In [49]:
print(hidden_layer_output)

Tensor("Sigmoid_2:0", shape=(6128, 730), dtype=float32)


In [74]:
#weights_output = tf.Variable(tf.truncated_normal(shape=[h_size, R.shape[1]], dtype=tf.float64))
weights_output = tf.Variable(tf.random_normal([R3.shape[0],R3.shape[0]],
                                       mean=0,
                                       stddev=0.01,
                                       name="weights"))
bias_output = tf.Variable(tf.random_normal([R2.shape[1],R3.shape[0]],
                                    mean=0,
                                    stddev=0.01,
                                    name="bias"))
#bias_output = tf.Variable(tf.truncated_normal(shape=[1,R.shape[1]], dtype=tf.float64))

In [75]:
af_input_output = tf.matmul(hidden_layer_output, weights_output)+bias_output

In [76]:
predictions = tf.nn.sigmoid(af_input_output)
print(predictions.T)

Tensor("Sigmoid_6:0", shape=(6128, 730), dtype=float32)


In [77]:
#prediction_error = 0.5 * tf.reduce_sum(tf.subtract(predictions, train_out) * tf.subtract(predictions, train_inp))
prediction_error = tf.nn.l2_loss(predictions-train_out, name="squared_error_cost")

ValueError: Dimensions must be equal, but are 730 and 6128 for 'sub_2' (op: 'Sub') with input shapes: [6128,730], [?,6128].

In [56]:
print(prediction_error)

Tensor("squared_error_cost_1:0", shape=(), dtype=float32)


In [57]:
numEpochs = 700

learningRate = tf.train.exponential_decay(learning_rate=0.0008,
                                          global_step= 1,
                                          decay_steps=new_R.shape[0],
                                          decay_rate= 0.95,
                                          staircase=True)

In [58]:
#train_op = tf.train.GradientDescentOptimizer(0.05).minimize(prediction_error)
train_op = tf.train.GradientDescentOptimizer(learningRate).minimize(prediction_error)

In [59]:
sess = tf.Session()

In [60]:
sess.run(tf.global_variables_initializer())

In [61]:
correct_predictions_OP = tf.equal(tf.argmax(predictions,1),tf.argmax(train_out,1))
accuracy_OP = tf.reduce_mean(tf.cast(correct_predictions_OP, "float"))

In [38]:
R2 = np.array(test_ratings.pivot(index='user_id',columns='movie_id',values='rating').fillna(0))
R2 = R2[:730,:]
R3 = R[:,:6128]
print(R2,R2.shape)
print(R3,R3.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 3. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]] (730, 6128)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [4. 3. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 4. ... 0. 0. 0.]] (730, 6128)


In [62]:
cost = 0
diff = 1
epoch_values = []
accuracy_values = []
cost_values = []

for i in range(numEpochs):
    if i > 1 and diff < .0001:
        print("change in cost %g; convergence."%diff)
        break
    else:
        step = sess.run(train_op, feed_dict={train_inp: R3, train_out: R2})
        if i % 10 == 0:
            epoch_values.append(i)
            train_accuracy, newCost = sess.run([accuracy_OP, prediction_error], feed_dict={train_inp: R3, train_out: R2})
            accuracy_values.append(train_accuracy)
            cost_values.append(newCost)
            diff = abs(newCost - cost)
            cost = newCost
            print("step %d, training accuracy %g, cost %g, change in cost %g"%(i, train_accuracy, newCost, diff))


# How well do we perform on held-out test data?
print("final accuracy on test set: %s" %str(sess.run(accuracy_OP, 
                                                     feed_dict={train_inp: R3, 
                                                                train_out: R2})))

InvalidArgumentError: Incompatible shapes: [6128,6128] vs. [730,6128]
	 [[node gradients_1/sub_1_grad/BroadcastGradientArgs (defined at <ipython-input-58-0a6bc32a1192>:2)  = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients_1/sub_1_grad/Shape, gradients_1/sub_1_grad/Shape_1)]]

Caused by op 'gradients_1/sub_1_grad/BroadcastGradientArgs', defined at:
  File "C:\Users\hp\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\hp\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\hp\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\hp\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\Users\hp\Anaconda3\li

In [ ]:
for step in range(500):
    op, err, p = sess.run(fetches=[train_op, prediction_error, predictions],
                          feed_dict={train_inp: new_R,train_out: R})
    if(step%100==0):
        print(str(step), ": ", err)

0 :  637287.25


In [ ]:
print("Predicted Output: ", sess.run(predict, feed_dict={train_inp: new_R}))
sess.close()